# Homework 1 

In this homework, we're going to download finance data from various sources and make simple calculations/analysis.

## Install YFinance package

!pip install yfinance

## Import necessary librraies

In [1]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import time
from datetime import date

### Question 1: [Macro] Average growth of GDP in 2023

**What is the average growth (in %) of GDP in 2023?**

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1). 
Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers).
Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer  5.7

In [2]:
# define start and end period
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-70, month=end.month, day=end.day)
print(f'Period for indexes: {start} to {end} ')

# read the data using pandas data reader from FRED
gdpc1 = pdr.DataReader("GDPC1", "fred", start=start)

Year = 2024; month= 4; day=24
Period for indexes: 1954-04-24 to 2024-04-24 


In [3]:
gdpc1.head()

,GDPC1
DATE,
1954-07-01,2880.482
1954-10-01,2936.852
1955-01-01,3020.746
1955-04-01,3069.910
1955-07-01,3111.379


In [4]:
gdpc1.tail()

,GDPC1
DATE,
2022-10-01,21989.981
2023-01-01,22112.329
2023-04-01,22225.350
2023-07-01,22490.692
2023-10-01,22679.255


In [5]:
# Shift index by number of periods =4 
# as mentioned in description of the question
gdpc1['gdpc1_us_yoy'] = gdpc1.GDPC1 / gdpc1.GDPC1.shift(4) - 1
gdpc1['gdpc1_us_yoy'] = round(gdpc1['gdpc1_us_yoy'] * 100, 1)

In [6]:
# Now measure average change of 2023 i.e values of 2023 year

# filter data to 2023
gdpc1_2023 = gdpc1.loc['2023']

# calculate mean
average_growth_2023 = gdpc1_2023['gdpc1_us_yoy'].mean()

print("The mean of yearly growth for 2023 = {0}%".format(round(average_growth_2023,1)))

The mean of yearly growth for 2023 = 2.5%


### Question 2. [Macro] Inverse "Treasury Yield"

**Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.**


Download DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2,
 https://fred.stlouisfed.org/series/DGS10). Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join()), calculate the difference dgs10-dgs2 daily.

(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)

In [7]:
start = '2000-01-01'

dgs2 = pdr.DataReader("DGS2", "fred", start=start)
dgs10 = pdr.DataReader("DGS10", "fred", start=start)

In [8]:
# merge two data frames
df_rates = dgs10.join(dgs2, how='inner', lsuffix='_dgs10', rsuffix='_dgs2')

In [9]:
# get daily diff
df_rates['diff'] = df_rates['DGS10'] - df_rates['DGS2']

# locate minimum value
min_diff_since_2000 = df_rates['diff'].min()

print("Minimum value of (dgs10 - dgs2) since 2000: {0}".format(round(min_diff_since_2000,1)))

Minimum value of (dgs10 - dgs2) since 2000: -1.1


For the market and investors, an inverted yield curve may indicate that the economy is heading towards or already in a recession

### Question 3. [Index] Which Index is better recently?

**Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)**

Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/). 

Compare 5Y growth for both (between 2019-04-09 and 2024-04-09). Select the higher growing index and write down the growth in % (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.

(Additional: think of other indexes and try to download stats and compare the growth? Do create 10Y and 20Y growth stats. What is an average yearly growth rate (CAGR) for each of the indexes you select?)

In [11]:
# define dates for start and end
start_date = '2019-04-09'
end_date = '2024-04-09'

# s and p 500 data
df_sandp = yf.download(tickers="^GSPC", start=start_date, end=end_date, progress=False)['Adj Close']
# get ipc  Mexico
df_ipc = yf.download(tickers="^MXX", start=start_date, end=end_date, progress=False)['Adj Close']

In [12]:
df_sandp.head()

Date
2019-04-09    2878.199951
2019-04-10    2888.209961
2019-04-11    2888.320068
2019-04-12    2907.409912
2019-04-15    2905.580078
Name: Adj Close, dtype: float64

In [13]:
df_ipc.head()

Date
2019-04-09    45151.628906
2019-04-10    44909.140625
2019-04-11    44580.058594
2019-04-12    44686.058594
2019-04-15    44625.781250
Name: Adj Close, dtype: float64

In [14]:
# ratio of start to finish
spx_growth_ratio = df_sandp.iloc[-1] / df_sandp.iloc[0]
ipc_growth_ratio = df_ipc.iloc[-1] / df_ipc.iloc[0]

spx_pct_increase = (spx_growth_ratio - 1) * 100
ipc_pct_increase = (ipc_growth_ratio - 1) * 100

In [15]:
# check with one of the two performed better
if spx_pct_increase > ipc_pct_increase:
    best = 'S&P 500'
    pct_increase = round(spx_pct_increase)
else:
    best = 'IPC Mexico'
    pct_increase = round(ipc_pct_increase)

print("The better performing index recently is {0} with a growth of {1} percent in previous 5 years".format(best,pct_increase))

The better performing index recently is S&P 500 with a growth of 81 percent in previous 5 years


### Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks


**Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023**


Download the 2023 daily OHLCV data on Yahoo Finance for top6 stocks on earnings (https://companiesmarketcap.com/most-profitable-companies/): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.

Here is the example data you should see in Pandas for "2222.SR": https://finance.yahoo.com/quote/2222.SR/history

Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value.
Round the result to two decimal places (e.g. 0.1575 will be 0.16)

(Additional: why this may be important for your research?)


In [16]:
# brands whose stoocks we are interested in
brands = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

# load the data for 52 weeks of 2023
dF_stocks = yf.download(brands, start="2023-01-01", end="2023-12-31", progress=False)

maximum_stock_value = dF_stocks['Adj Close'].max()
minimum_stock_value = dF_stocks['Adj Close'].min()

In [17]:
# get the required ratio
ratios = ((maximum_stock_value - minimum_stock_value) / maximum_stock_value).round(2)

# now get index of brand with highest ratio
max_range_ratio_stock = ratios.idxmax()
# and its value
max_range_ratio_value = ratios.max()

print("{0} had highest ratio in 2023 of {1}".format(max_range_ratio_stock,max_range_ratio_value))

MSFT had highest ratio in 2023 of 0.42


### Question 5. [Stocks] Dividend Yield
**Find the largest dividend yield for the same set of stocks**

Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023.
You can use `get_actions()` method or `.dividends` field in yfinance library (https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start)

Sum up all dividends paid in 2023 per company and divide each value by the closing price (Adj.Close) at the last trading day of the year.

Find the maximum value in % and round to 1 digit after the decimal point. (E.g., if you obtained $1.25 dividends paid and the end year stock price is $100, the dividend yield is 1.25% -- and your answer should be equal to 1.3)

In [18]:
# brands whose stoocks we are interested in
brands = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

# get the data
df_div = yf.download(brands, start="2023-01-01", end="2023-12-31", progress=False, actions=True)

# total dividends for every company
div_sum = df_div['Dividends'].sum()

# end of year final stock price
final_price = df_div.iloc[-1]['Adj Close']

# yield on dividends
yield_on_divs = (div_sum / final_price) * 100

# max yield
max_yield = yield_on_divs.max()

print("Max dividend yield for a company for 2023 was {0}%".format(round(max_yield,1)))

Max dividend yield for a company for 2023 was 2.4%


### Question 6. [Exploratory] Investigate new metrics

**Free text answer**

Download and explore a few additional metrics or time series that might be valuable for your project and write down why (briefly).



---
### Question 7. [Exploratory] Time-driven strategy description around earnings releases

**Free text answer**

Explore earning dates for the whole month of April - e.g. using YahooFinance earnings calendar (https://finance.yahoo.com/calendar/earnings?from=2024-04-21&to=2024-04-27&day=2024-04-23). Compare with the previous closed earnings (e.g., recent dates with full data https://finance.yahoo.com/calendar/earnings?from=2024-04-07&to=2024-04-13&day=2024-04-08). 

Describe an analytical strategy/idea (you're not required to implement it) to select a subset companies of interest based on the future events data.